In [1]:
import threading

def print_numbers():
    for i in range(10):
        print(i)

def print_letters():
    for letter in 'abcdefghij':
        print(letter)

# Create threads
thread1 = threading.Thread(target=print_numbers)
thread2 = threading.Thread(target=print_letters)

# Start threads
thread1.start()
thread2.start()

# Join threads back to the parent process, which is this program
thread1.join()
thread2.join()


0
1
2
3
4
5
6
7
8
9
a
b
c
d
e
f
g
h
i
j



Multithreading is a way to concurrently run multiple threads. Threads are smaller units of a process. In Python, threading is mostly used for I/O-bound tasks, such as downloading files, reading from disk, or network operations, because Python threads are not well-suited for CPU-bound tasks due to the Global Interpreter Lock (GIL).

In this example, print_numbers and print_letters are each run in their own thread. This means they will execute concurrently, leading to a mixed output of numbers and letters. The order of the output will likely be different each time the program is run because the threads are being scheduled to run by the operating system and may not start or finish at exactly the same time.

You should be aware that multithreading in Python doesn't actually result in true parallel execution due to the Global Interpreter Lock (GIL). Python's GIL is a mutex (or a lock) that allows only one thread to execute at a time in a single process. Python uses the GIL to simplify memory management and to avoid conflicts among threads.

For CPU-bound tasks where you do need parallelism, you should use multiprocessing (which involves separate processes, each with its own Python interpreter and its own GIL) instead of multithreading. You might also consider using a native extension such as Numba or Cython, or a different language that supports true multithreading.

Keep in mind that multithreading can make a program more complex and more difficult to debug. It can lead to issues with shared state (where one thread changes a value another thread was expecting) and can cause race conditions, where the behavior of your program depends on the exact timing of events, which is generally not predictable. Before using multithreading, it's worth considering if there's a way to achieve the same result with simpler code.

Creating multiple threads and passing arguments to the target function:

In [2]:
import threading

def print_message(message, times):
    for i in range(times):
        print(message)

# Create threads
thread1 = threading.Thread(target=print_message, args=("Hello", 10))
thread2 = threading.Thread(target=print_message, args=("World", 10))

# Start threads
thread1.start()
thread2.start()

# Join threads back to the parent process
thread1.join()
thread2.join()


Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
World
World
World
World
World
World
World
World
World
World


In this example, args is used to pass arguments to the target function.

Using threads to speed up I/O-bound tasks (such as downloading web pages):

In [3]:
import requests
import threading
import time

urls = [
    "http://www.google.com",
    "http://www.microsoft.com",
    "http://www.amazon.com",
    # Add more URLs as needed
]

def fetch_url(url):
    response = requests.get(url)
    print(f"Got response {response.status_code} from {url}")

start_time = time.time()

threads = []
for url in urls:
    thread = threading.Thread(target=fetch_url, args=(url,))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

end_time = time.time()
print(f"Downloaded {len(urls)} in {end_time - start_time} seconds")


Got response 200 from http://www.microsoft.com
Got response 200 from http://www.google.com
Got response 503 from http://www.amazon.com
Downloaded 3 in 0.43097758293151855 seconds


This program starts a new thread for each URL to be downloaded, which can speed up the total download time because downloads can happen concurrently.

Multithreading with a subclass of Thread:
In addition to passing a function to the Thread constructor, as shown in previous examples, you can also subclass Thread and override its run method.

In [4]:
import threading
import time

class SleeperThread(threading.Thread):
    def __init__(self, id, sleep_time):
        super().__init__()
        self.id = id
        self.sleep_time = sleep_time

    def run(self):
        print(f"Thread {self.id} is going to sleep for {self.sleep_time} seconds.")
        time.sleep(self.sleep_time)
        print(f"Thread {self.id} has woken up.")

# Create threads
threads = []
for i in range(5):
    threads.append(SleeperThread(i, i))

# Start threads
for thread in threads:
    thread.start()

# Join threads back to the parent process
for thread in threads:
    thread.join()


Thread 0 is going to sleep for 0 seconds.
Thread 0 has woken up.
Thread 1 is going to sleep for 1 seconds.
Thread 2 is going to sleep for 2 seconds.
Thread 3 is going to sleep for 3 seconds.
Thread 4 is going to sleep for 4 seconds.
Thread 1 has woken up.
Thread 2 has woken up.
Thread 3 has woken up.
Thread 4 has woken up.


This example defines a SleeperThread class that sleeps for a specified amount of time. Each thread is given a different sleep time, so they wake up at different times.

Synchronizing threads with Lock:
If you have multiple threads that need to modify shared state, you can use a Lock to ensure that only one thread modifies the state at a time. Here's an example where two threads increment a counter:

In [5]:
import threading

class Counter:
    def __init__(self):
        self.value = 0
        self.lock = threading.Lock()

    def increment(self):
        with self.lock:
            self.value += 1

def worker(counter):
    for _ in range(10000):
        counter.increment()

counter = Counter()
threads = []

# Start threads
for _ in range(2):
    thread = threading.Thread(target=worker, args=(counter,))
    thread.start()
    threads.append(thread)

# Join threads back to the parent process
for thread in threads:
    thread.join()

print(counter.value)


20000


In this example, the increment method uses a Lock to prevent two threads from modifying self.value at the same time. If you run this program, you should see that the final value of the counter is 20,000, as expected. If you removed the lock, the final value could be less than 20,000 due to race conditions.